# Finding the frauds

### This Ipython notebook aims to find out all the likely fraudsters based on certain features and user behavior.

In [12]:
import pandas as pd
import os
from file import *

### Loading the data from csv files to dataframes using pandas

In [13]:
currency_df = pd.read_csv(os.path.join(dir_path, './fct_data/currency_details.csv'))
fx_df = pd.read_csv(os.path.join(dir_path, './fct_data/fx_rates.csv'))
trans_df = pd.read_csv(os.path.join(dir_path, './fct_data/transactions.csv'), index_col=0)
users_df = pd.read_csv(os.path.join(dir_path, './fct_data/users.csv'), index_col=0)
fraudster_df = pd.read_csv(os.path.join(dir_path, './fct_data/fraudsters.csv'), index_col=0)


Converting the column headers to lowercase

In [14]:
currency_df.columns = map(str.lower, currency_df.columns)
fx_df.columns = map(str.lower, fx_df.columns)
trans_df.columns = map(str.lower, trans_df.columns)
users_df.columns = map(str.lower, users_df.columns)

# Explore the data
trans_df.head()

,currency,amount,state,created_date,merchant_category,merchant_country,entry_method,user_id,type,source,id
0,GBP,3738,COMPLETED,2015-10-11 09:05:43.016000,bar,AUS,misc,7285c1ec-31d0-4022-b311-0ad9227ef7f4,CARD_PAYMENT,GAIA,5a9ee109-e9b3-4598-8dd7-587591e6a470
1,GBP,588,COMPLETED,2015-10-11 20:08:39.150000,NaN,CA,misc,20100a1d-12bc-41ed-a5e1-bc46216e9696,CARD_PAYMENT,GAIA,28d68bf4-460b-4c8e-9b95-bcda9ab596b5
2,GBP,1264,COMPLETED,2015-10-11 11:37:40.908000,NaN,UKR,misc,0fe472c9-cf3e-4e43-90f3-a0cfb6a4f1f0,CARD_PAYMENT,GAIA,1f1e8817-d40b-4c09-b718-cfc4a6f211df
3,GBP,66,REVERTED,2015-10-11 20:08:35.310000,NaN,CA,misc,20100a1d-12bc-41ed-a5e1-bc46216e9696,CARD_PAYMENT,GAIA,a7aaf78c-d201-456f-9e6d-612a795e8c32
4,GBP,968,COMPLETED,2015-10-11 02:46:47.640000,supermarket,NZL,misc,821014c5-af06-40ff-91f4-77fe7667809f,CARD_PAYMENT,GAIA,27dd99a2-5539-4ba9-876a-1a94abc2701f


### Approach to find the fraudsters

Looking at the data, you must first list down all the potential features which are crucial in the process of a transaction. Listing down all the important factors which can provide some leads:
1. KYC - Legal Identification status of the user
2. has_email - if the user has a registered email in our database
3. state - Is he active or passive or locked?
4. Failed sign in attempts - In case of a hacked account
5. Merchant country - Look at the country to which the amount is being transferred

Now, we need to build a potential scenario by knitting together all the above parameters.
We are basically testing the user's authenticity from all the parameters mentioned above.

Logic behind declaring someone as a Fraudster:
- If a user has its KYC failed with no email address registered and is actively using our platform.
- Above this, check if the country is recorded in the transaction to which the transfer has been made.

In [19]:
# finding out all the unidentified users who have their KYC failed
unidentified_df = users_df.query('kyc == "FAILED" and has_email == 0 and state == "ACTIVE"')

# merging(joining) the above dataframe with the transactional database on user_id
unidentified_txns = pd.merge(unidentified_df, trans_df, left_on='id', right_on='user_id')

unidentified_txns

,failed_sign_in_attempts,kyc,birth_year,country,state_x,created_date_x,terms_version,phone_country,has_email,id_x,...,amount,state_y,created_date_y,merchant_category,merchant_country,entry_method,user_id,type,source,id_y
0,0,FAILED,1966,GB,ACTIVE,2015-11-06 12:39:34.355000,NaN,GB||JE||IM||GG,0,62edb2a5-8b28-466a-9a9a-b3e314f2f771,...,1000,DECLINED,2015-11-17 20:30:33.796000,NaN,NaN,misc,62edb2a5-8b28-466a-9a9a-b3e314f2f771,ATM,GAIA,9d99f7f9-ba47-42e4-8151-a5a2aef27f96
1,0,FAILED,1993,GE,ACTIVE,2015-10-18 07:02:26.487000,NaN,GE,0,f6b7f148-bb9f-42c9-9c56-9243f83ee3d5,...,1000,FAILED,2015-11-10 20:26:34.738000,NaN,NaN,misc,f6b7f148-bb9f-42c9-9c56-9243f83ee3d5,TOPUP,NYX,d659b109-0509-49b1-af70-b9a2158b33a4
2,0,FAILED,1981,GB,ACTIVE,2015-08-09 07:33:29.491000,NaN,GB||JE||IM||GG,0,82a2ecd2-0b94-4629-85c0-141d7af04e5f,...,2585,COMPLETED,2015-09-24 11:31:29.321000,NaN,ITA,misc,82a2ecd2-0b94-4629-85c0-141d7af04e5f,CARD_PAYMENT,GAIA,e60174ba-59d1-4084-bcbe-43152f7d22e4
3,0,FAILED,1981,GB,ACTIVE,2015-08-09 07:33:29.491000,NaN,GB||JE||IM||GG,0,82a2ecd2-0b94-4629-85c0-141d7af04e5f,...,3320,COMPLETED,2015-09-27 08:25:57.976000,NaN,ITA,misc,82a2ecd2-0b94-4629-85c0-141d7af04e5f,CARD_PAYMENT,GAIA,74d947bd-49e9-42f4-ba74-e18d86fa29d9
4,0,FAILED,1981,GB,ACTIVE,2015-08-09 07:33:29.491000,NaN,GB||JE||IM||GG,0,82a2ecd2-0b94-4629-85c0-141d7af04e5f,...,5535,COMPLETED,2015-09-27 14:18:17.138000,point_of_interest,GBR,misc,82a2ecd2-0b94-4629-85c0-141d7af04e5f,CARD_PAYMENT,GAIA,65c7a581-6cbd-4202-9666-f67791ae6c7d
5,0,FAILED,1981,GB,ACTIVE,2015-08-09 07:33:29.491000,NaN,GB||JE||IM||GG,0,82a2ecd2-0b94-4629-85c0-141d7af04e5f,...,10624,COMPLETED,2015-09-26 12:02:31.562000,NaN,ITA,misc,82a2ecd2-0b94-4629-85c0-141d7af04e5f,CARD_PAYMENT,GAIA,29fca723-8a58-4660-bc54-465d553436ca
6,0,FAILED,1981,GB,ACTIVE,2015-08-09 07:33:29.491000,NaN,GB||JE||IM||GG,0,82a2ecd2-0b94-4629-85c0-141d7af04e5f,...,1362,COMPLETED,2015-09-25 16:21:11.801000,NaN,ITA,misc,82a2ecd2-0b94-4629-85c0-141d7af04e5f,CARD_PAYMENT,GAIA,2a05136d-1bf8-469c-85c5-53cbbda5af21
7,0,FAILED,1981,GB,ACTIVE,2015-08-09 07:33:29.491000,NaN,GB||JE||IM||GG,0,82a2ecd2-0b94-4629-85c0-141d7af04e5f,...,3537,COMPLETED,2015-09-25 17:56:50.974000,NaN,ITA,misc,82a2ecd2-0b94-4629-85c0-141d7af04e5f,CARD_PAYMENT,GAIA,f4a1c88d-0f8d-4e43-bcef-4b6a053165a6
8,0,FAILED,1981,GB,ACTIVE,2015-08-09 07:33:29.491000,NaN,GB||JE||IM||GG,0,82a2ecd2-0b94-4629-85c0-141d7af04e5f,...,2066,COMPLETED,2015-09-26 12:12:18.467000,NaN,ITA,misc,82a2ecd2-0b94-4629-85c0-141d7af04e5f,CARD_PAYMENT,GAIA,1fcd497c-b702-442c-a062-4598991b163f
9,0,FAILED,1981,GB,ACTIVE,2015-08-09 07:33:29.491000,NaN,GB||JE||IM||GG,0,82a2ecd2-0b94-4629-85c0-141d7af04e5f,...,9591,COMPLETED,2015-09-26 06:03:14.521000,NaN,ITA,misc,82a2ecd2-0b94-4629-85c0-141d7af04e5f,CARD_PAYMENT,GAIA,e6bf4bbd-ee06-4644-b389-19d733f7faea


Among the unidentified users, we will check for their transactional data to check for the merchant country. If the country to which the transfer has been made is captured. Narrowing down the list of users who are most likely to be frauds.

In [16]:
# checking for NaN values in the merchant_country column in the above obtained dataframe
frauds_df = unidentified_txns[unidentified_txns['merchant_country'].isnull()]

### Removing all the already obtained fraudsters in the list

In [17]:
potential_fraudsters = set(frauds_df['id_x']) - set(fraudster_df['user_id'])

In [18]:
# Here is the final list of potential fraudsters on our platform.
potential_fraudsters

{'146cb748-d9a7-410a-b2ec-f5c403f7149d',
 '16102a27-534e-48d7-b7d2-99e08e6c7597',
 '1eb7c725-f748-4e76-98cc-1bb6b58d4e4a',
 '5302559e-9dbd-4f1d-ba3b-4e5a10f0e51a',
 '5ff27f1c-65e2-426f-a139-56d7cf6f4a42',
 '62edb2a5-8b28-466a-9a9a-b3e314f2f771',
 '78a0cfa2-f9d3-4f07-94c4-a97f76595589',
 '82a2ecd2-0b94-4629-85c0-141d7af04e5f',
 'acbb5850-8df2-4614-a277-83e020a9edc9',
 'c876e9f6-c0f8-4ee6-80a0-2a9ef751cc80',
 'ccc08947-8246-45c3-ae88-8f9327b60a74',
 'd1e9aea4-e0e3-4ac3-af74-f8095c8b8861',
 'd4aee110-f821-4545-b617-1f49eacad2b9',
 'dd8e1dd7-e467-41ff-908f-4e0ab64d9af3',
 'e0ff10b4-4e05-45c8-9183-ab45ae37913e',
 'e14f2def-d412-4ba6-90da-6ff8c8bf2782',
 'f6b7f148-bb9f-42c9-9c56-9243f83ee3d5'}

### Now, among these likely fraudsters, we'll shortlist a few high potential fraudsters who are making transfers of amount more than 20000.

In [40]:
high_pot_frauds_df = frauds_df[frauds_df['amount'] > 20000]

In [41]:
high_pot_fraudsters = set(high_pot_frauds_df['id_x']) - set(fraudster_df['user_id'])

### Ids of the high risk users.

In [43]:
high_pot_fraudsters

{'146cb748-d9a7-410a-b2ec-f5c403f7149d',
 '16102a27-534e-48d7-b7d2-99e08e6c7597',
 '82a2ecd2-0b94-4629-85c0-141d7af04e5f',
 'acbb5850-8df2-4614-a277-83e020a9edc9',
 'dd8e1dd7-e467-41ff-908f-4e0ab64d9af3'}

## Financial crime Typology likely to be prevalent here

We have clearly focused on the legal identification of the user to be able to transact the money.

Users who don't have their KYC done, we don't have any record or method to:
- source of income
- assess further potential risks like money laundering and other internet sourced frauds.
- make sure that all our clients are anti-bribery compliant
- to put in place an anti-corruption due diligence 

Keeping in mind the above mentioned crucial factors, here is how we can conduct due diligence on these high risk users:
1. Revoking their permission to make any transaction before they get their KYC PASSED.
2. For an amount greater than 20000, Merchant's country should be made mandatory to be supplied.
3. Email should be verified along with the phone number.
4. All the tax reciepts / PAN(Anti-corruption information) card details should be verified. 
